In [41]:
import requests
import collections
import re
from datetime import date
import datetime,time
from dateutil.rrule import rrule, DAILY, HOURLY
import pandas as pd
from utils import *
import sys

#sys.path.append('/usr/local/lib/python2.7/site-packages/')

Check quota [here](https://console.developers.google.com/project/childmarriage-1019/apiui/apiview/youtube/quotas)

In [76]:
taxonomyWords=['child marriage','child bride','underage bride','teen bride','forced marriage','early marriage']
taxonomyWords.extend([u'बाल विवाह',u'बालिका वधु',u'नाबालिक विवाह',u'नाबालिक वधु',u'किशोर वधु',u'किशोर बीवी',u'जबरन शादी',u'जबरन विवाह'])
taxonomyWords=map(lambda x:'"'+x+'"',taxonomyWords)
FULLQUERY='|'.join(taxonomyWords)

In [77]:
FULLQUERY

u'"child marriage"|"child bride"|"underage bride"|"teen bride"|"forced marriage"|"early marriage"|"\u092c\u093e\u0932 \u0935\u093f\u0935\u093e\u0939"|"\u092c\u093e\u0932\u093f\u0915\u093e \u0935\u0927\u0941"|"\u0928\u093e\u092c\u093e\u0932\u093f\u0915 \u0935\u093f\u0935\u093e\u0939"|"\u0928\u093e\u092c\u093e\u0932\u093f\u0915 \u0935\u0927\u0941"|"\u0915\u093f\u0936\u094b\u0930 \u0935\u0927\u0941"|"\u0915\u093f\u0936\u094b\u0930 \u092c\u0940\u0935\u0940"|"\u091c\u092c\u0930\u0928 \u0936\u093e\u0926\u0940"|"\u091c\u092c\u0930\u0928 \u0935\u093f\u0935\u093e\u0939"'

In [60]:
QUERY=re.escape('"child marriage"|"child bride"')
from secrets import KEY

diff=1

In [114]:
def getCommentsFromVideo(vid):
    requestString='https://www.googleapis.com/youtube/v3/commentThreads?part=snippet&maxResults=100&videoId='+vid+'&key='+KEY

    res=requests.get(requestString)
    commentData=res.json()

    if res.status_code==403:
        # Comments disabled
        return [],None
    
    isMore=False
    
    nextToken=commentData.get('nextPageToken')
    
    if nextToken:
        if len(nextToken)>0:
            print '!!!!More than 100 comments....'
            isMore=True
    
    try:
        print 'Got %d comments' % (len(commentData['items']))
    except:
        print '!!!!ERROR',commentData.keys()
        print res.status_code
        print commentData
    
    return commentData['items'],isMore

In [81]:
def getCountryFromInfo(info):
    if len(info['items'])>0:
        return info['items'][0]['snippet'].get('country')
    else:
        return None

In [82]:
def getDescriptionFromInfo(info):
    if len(info['items'])>0:
        return info['items'][0]['snippet'].get('description')
    else:
        return None

In [88]:
def getTitleFromInfo(info):
    if len(info['items'])>0:
        return info['items'][0]['snippet'].get('title')
    else:
        return None

In [18]:
def getVideoInfoFromVideo(vid):

        requestString='https://www.googleapis.com/youtube/v3/videos?part=snippet,statistics&id='+vid+'&key='+KEY

        print 'QUERYING VIDEO',vid

        d=requests.get(requestString)
#        print d.status_code
        
#        print d.json()['items'][0].keys()

        return d.json()

In [19]:
def getChannelInfoFromChannel(cid):
    requestString='https://www.googleapis.com/youtube/v3/channels?part=snippet&id='+cid+'&key='+KEY

    print 'QUERYING CHANNEL',cid

    d=requests.get(requestString)
#    print d.status_code

    return d.json()

In [106]:
def getRepliesFromComment(commentId):
    requestString='https://www.googleapis.com/youtube/v3/comments?part=snippet&maxResults=100&parentId='+commentId+\
    '&key='+KEY
    
    d=requests.get(requestString)
    
    return d.json()

In [115]:
nDayDiff=10
# Look back n days from start date

diff=1
# Start date is yesterday

now=datetime.datetime.now()

endDate=date(now.year,now.month,now.day)
# Throw away hour, minute, second

endDate=endDate-datetime.timedelta(days=diff)
endDateTemp=endDate
# Look back diff days

startDate=endDate-datetime.timedelta(days=nDayDiff)
startDateTemp=startDate

endDate=endDate.strftime("%Y-%m-%dT%H:%M:%SZ")
startDate=startDate.strftime("%Y-%m-%dT%H:%M:%SZ")

startDates=[startDate]
endDates=[endDate]

print startDates,endDates


startDates=[d for d in rrule(DAILY,dtstart=startDateTemp,until=endDateTemp-datetime.timedelta(days=1))]
startDates=[d.strftime("%Y-%m-%dT%H:%M:%SZ") for d in startDates]

print startDates

#sys.exit(1)

vIdList=[]
vTimeList=[]
vTitleList=[]
vLangList=[]
vTagList=[]
vDescriptionList=[]
vCommentCountList=[]
vViewCountList=[]
vFavouriteCountList=[]
vDislikeCountList=[]
vLikeCountList=[]

cIdList=[]
cCountryList=[]
cDescriptionList=[]
cTitleList=[]

commentIdList=[]
commentVideoIdList=[]
commentTextList=[]
commentUserIdList=[]
commentLikesList=[]
commentTimeList=[]

# commentCount:0 viewCount:4 favoriteCount:0 dislikeCount:0 likeCount:0

##############
for start,end in zip(startDates[0:-1],startDates[1:]):

    requestString='https://www.googleapis.com/youtube/v3/search?part=snippet&q='+FULLQUERY+'&key='+KEY+'&maxResults=50&type=video&publishedBefore='+end+'&publishedAfter='+start

    print 'QUERYING API IN RANGE',start,end

    d=requests.get(requestString)
    print d.status_code

    data=d.json()
    
    print data.keys()
    
    print 'Got %d videos' % (len(data['items']))

    videoIds=[]

    for v,video in enumerate(data['items']):
        
        ############################################
        # Video stuff

        videoId=video[u'id']['videoId']
        vIdList.append(videoId)
        
        videoIds.append(videoId)
        videoData=getVideoData(videoId)

        print videoId
        print '\t',videoData['items'][0].keys()
#        for part in videoData['items']:
#            print part['kind'],part['etag']
#        print videoData['items'][0]
        res=getVideoInfoFromVideo(videoData['items'][0]['id'])
        channelId=res['items'][0]['snippet']['channelId']

        vTime,vTitle,vTags,vLang,vDescription,vCommentCount,vViewCount,vFavouriteCount,vDislikeCount,vLikeCount=\
        getVideoParts(res['items'][0])
        
        print 'Video Title:',vTitle
        vTitleList.append(vTitle)
        
        print 'Video Time',vTime
        vTimeList.append(vTime)
        
        print 'Video Tags:',vTags
        vTagList.append(vTags)
        
        print 'Video Language:',vLang
        vLangList.append(vLang)
        
        if False:print 'Video Description:',vDescription
        vDescriptionList.append(vDescription)
        
        print 'Video Stats',' '.join([k+':'+v for k,v in zip(['comments','views','favs','dislike','like'],[vCommentCount,vViewCount,vFavouriteCount,vDislikeCount,vLikeCount]) if v])
        vCommentCountList.append(vCommentCount)
        vViewCountList.append(vViewCount)
        vFavouriteCountList.append(vFavouriteCount)
        vDislikeCountList.append(vDislikeCount)
        vLikeCountList.append(vLikeCount)
        
        commentThreads,isMore=getCommentsFromVideo(videoId)
        
        for thread in commentThreads:
            
            commentId=thread['snippet']['topLevelComment']['id']
            commentIdList.append(commentId)
            
            commentVideoId=thread['snippet']['topLevelComment']['snippet']['videoId']
            commentVideoIdList.append(commentVideoId)
            
            commentText=thread['snippet']['topLevelComment']['snippet']['textDisplay']
            commentTextList.append(commentText)
            
            commentTime=thread['snippet']['topLevelComment']['snippet']['publishedAt']
            commentTimeList.append(commentTime)
            
            commentUser=thread['snippet']['topLevelComment']['snippet']['authorChannelId']['value']
            commentUserIdList.append(commentUser)
            
            commentLikes=thread['snippet']['topLevelComment']['snippet']['likeCount']
            commentLikesList.append(commentLikes)
                    
            if thread['snippet']['totalReplyCount']>0:
                replies=getRepliesFromComment(commentId)
                print '%d replies' % (len(replies))
                sys.exit(1)
                
        ############################################
        # Channel stuff
        
        info=getChannelInfoFromChannel(channelId)
        cIdList.append(channelId)

        cCountry=getCountryFromInfo(info)
        print 'Channel Country:',cCountry
        cCountryList.append(cCountry)
        
        cDescription=getDescriptionFromInfo(info)
        if False:print 'Channel Description:',cDescription
        cDescriptionList.append(cDescription)
        
        cTitle=getTitleFromInfo(info)
        print 'Channel Title',cTitle
        cTitleList.append(cTitle)
        
        print '\n'

videoDf=pd.DataFrame(data={'id':vIdList,'title':vTitleList,'tags':vTagList,'lang':vLangList,'description':vDescriptionList,\
                'views':vViewCountList,'favourites':vFavouriteCountList,'dislikes':vDislikeCountList,\
                'likes':vLikeCountList,'channelId':cIdList,'channelCountry':cCountryList,\
                'channelDescription':cDescriptionList,'channelTitle':cTitleList},index=vTimeList)       
      
commentDf=pd.DataFrame(data={'text':commentTextList,'videoId':commentVideoIdList,'id':commentIdList,\
                             'likes':commentLikesList,'user':commentUserIdList},index=commentTimeList)
#print videoData

#for part in videoData['items']:
#    print part['kind'],part['etag']


['2015-07-25T00:00:00Z'] ['2015-08-04T00:00:00Z']
['2015-07-25T00:00:00Z', '2015-07-26T00:00:00Z', '2015-07-27T00:00:00Z', '2015-07-28T00:00:00Z', '2015-07-29T00:00:00Z', '2015-07-30T00:00:00Z', '2015-07-31T00:00:00Z', '2015-08-01T00:00:00Z', '2015-08-02T00:00:00Z', '2015-08-03T00:00:00Z']
QUERYING API IN RANGE 2015-07-25T00:00:00Z 2015-07-26T00:00:00Z
200
[u'nextPageToken', u'items', u'kind', u'etag', u'pageInfo']
Got 50 videos
fVaKzmvEtFQ
	[u'kind', u'etag', u'id']
QUERYING VIDEO fVaKzmvEtFQ
Video Title: Balika Vadhu – 05th August 2015 - बालिका वधु - Full Episode (HD)
Video Time 2015-07-25T15:23:32.000Z
Video Tags: None
Video Language: None
Video Stats comments:0 views:8947 favs:0 dislike:3 like:5
Got 0 comments
QUERYING CHANNEL UCd6P7-Ae8CvcwqVDkwT5fFg
Channel Country: None
Channel Title colors tv


PzzUYxvJmXk
	[u'kind', u'etag', u'id']
QUERYING VIDEO PzzUYxvJmXk
Video Title: DCLG funded project raising awareness of Forced Marriage and Honour Based Violence
Video Time 2015-07-25T15

SystemExit: 1

To exit: use 'exit', 'quit', or Ctrl-D.


In [102]:
videoDf.shape

(50, 13)

In [103]:
videoDf

,channelCountry,channelDescription,channelId,channelTitle,description,dislikes,favourites,id,lang,likes,tags,title,views
2015-08-02T02:12:04.000Z,None,Screams from the Netherworld of Fandom. The Le...,UCaRBVYI8HheW_P4HAiOK57A,DiamandaHagan,A sequel to the 6 possession episodes that had...,0,0,dpT6ldHYh-A,None,15,"[The Island of Doctor Moreau, Diamanda, Hagan,...",The Island of Doctor Moreau Review (with Oanci...,210
2015-08-02T07:27:37.000Z,None,,UC3umARMHCYbtpaiQNGeFSeg,Prashant Kumar,Balika vadhu 1st august 2015\nSet in rural Raj...,2,0,Knkb_HSNtSo,None,2,"[Balika vadhu 1st august 2015, Balika vadhu, B...",Balika vadhu 1st august 2015,1079
2015-08-02T19:37:27.000Z,None,,UCe3izJnI-GAZcWnrhlr7OYw,gon tv,'Balika Vadhu' traces the arduous journey of c...,0,0,T83KFJhl6K0,None,1,"[Balika, Vadhu, बालिका, वधु, Full, Episode, (HD)]",Balika Vadhu - 05th August 2015 - बालिका वधु -...,134
2015-08-02T19:37:24.000Z,None,,UCe3izJnI-GAZcWnrhlr7OYw,gon tv,'Balika Vadhu' traces the arduous journey of c...,0,0,h6es9DJ-Ga8,None,0,"[Balika, Vadhu, बालिका, वधु, Full, Episode, (HD)]",Balika Vadhu - 05th August 2015 - बालिका वधु -...,25
2015-08-02T19:37:48.000Z,None,,UCe3izJnI-GAZcWnrhlr7OYw,gon tv,'Balika Vadhu' traces the arduous journey of c...,0,0,kRZtSHx_Ps0,None,0,"[Balika, Vadhu, बालिका, वधु, Full, Episode, (HD)]",Balika Vadhu - 04th August 2015 - बालिका वधु -...,25
2015-08-02T19:37:17.000Z,None,,UCe3izJnI-GAZcWnrhlr7OYw,gon tv,'Balika Vadhu' traces the arduous journey of c...,2,0,c5cdhJV1I_8,None,0,"[Balika, Vadhu, बालिका, वधु, Full, Episode, (HD)]",Balika Vadhu - 05th August 2015 - बालिका वधु -...,45
2015-08-02T19:37:31.000Z,None,,UCe3izJnI-GAZcWnrhlr7OYw,gon tv,'Balika Vadhu' traces the arduous journey of c...,0,0,WwQPqlvI6uE,None,0,"[Balika, Vadhu, बालिका, वधु, Full, Episode, (HD)]",Balika Vadhu - 05th August 2015 - बालिका वधु -...,17
2015-08-02T02:39:58.000Z,None,,UCAJ6fJiKk92VXkLWUTLZdXA,Colors TV,"Published on Aug 1, 2015\nSubscribe to Colors ...",0,0,zMp4Y6BvMWw,None,0,"[Balika, Vadhu, 1st, August, 2015, बालिका, वधु...",Balika Vadhu - 5th August 2015 - बालिका वधु - ...,23
2015-08-02T19:37:44.000Z,None,,UCe3izJnI-GAZcWnrhlr7OYw,gon tv,'Balika Vadhu' traces the arduous journey of c...,0,0,rsdo2wdQHpc,None,0,"[Balika, Vadhu, बालिका, वधु, Full, Episode, (HD)]",Balika Vadhu - 04th August 2015 - बालिका वधु -...,8
2015-08-02T19:37:46.000Z,None,,UCe3izJnI-GAZcWnrhlr7OYw,gon tv,'Balika Vadhu' traces the arduous journey of c...,0,0,UJoSyzCIUAI,None,0,"[Balika, Vadhu, बालिका, वधु, Full, Episode, (HD)]",Balika Vadhu - 04th August 2015 - बालिका वधु -...,13


In [104]:
commentDf

,id,likes,text,user,videoId
2015-08-02T23:38:22.686Z,z13tedcx5xe3frouk23xzdxbvuikw3qw004,0,I don&#39;t care how many bad movies shes been...,UClxRLE4HmUfnI-CcKgzgepQ,dpT6ldHYh-A
2015-08-02T17:50:04.433Z,z12ctr5itua2ttf4v04ch14zesnqvhx4wkg0k,0,This was hell to film but IMHO it&#39;s one of...,UCj1sANqq2QgOuo3XeIAUu8A,dpT6ldHYh-A
2015-08-02T16:57:48.532Z,z12yyn5oqzqnjbvnu23icjah1uqnfjlaz04,0,Definitely my favourite Channel Awesome collab...,UCEEZw5HawqQ8tRHYal5R-FA,dpT6ldHYh-A
2015-08-02T13:35:13.060Z,z13rgdjovvupi1ory04cfzlorqitstwalw40k,0,Great crossover!<br /><br />RIP JewWario. He w...,UC4cWe1t4HvDxYkjF-WgBLfQ,dpT6ldHYh-A
2015-08-02T09:23:27.856Z,z12wtxnxbzq2szm3g220ejspbxadtnjwh04,0,"Dear Hagan, why are you so awesome? :3﻿",UC3Y1Lk2NScLm-oPSToLgNqA,dpT6ldHYh-A
2015-08-02T02:53:46.035Z,z13ue3jypon5zd1k522uwrdbiyq0yfbez,2,I always loved this one. Such a great collabor...,UCrd-aVEMKT7MLA0nCkrRjcA,dpT6ldHYh-A
2015-08-02T02:35:46.975Z,z12pwtmz2vatcjr0423hv3diivfeh3gwt04,0,Hello Diamanda!﻿,UCVV91_dDq-ea7XDi9WPI0ug,dpT6ldHYh-A


In [56]:
def getVideoParts(vData):
    title=res['items'][0]['snippet'].get('title')
    time=res['items'][0]['snippet'].get('publishedAt')
    tags=res['items'][0]['snippet'].get('tags')
    lang=res['items'][0]['snippet'].get('defaultAudioLanguage')
    description=res['items'][0]['snippet'].get('description')
    commentCount=viewCount=favouriteCount=dislikeCount=likeCount=None
    
    stats=res['items'][0].get('statistics')

    commentCount=stats.get('commentCount')
    viewCount=stats.get('viewCount')
    favouriteCount=stats.get('favoriteCount')
    dislikeCount=stats.get('dislikeCount')
    likeCount=stats.get('likeCount')
    
    return time,title,tags,lang,description,commentCount,viewCount,favouriteCount,dislikeCount,likeCount

In [8]:
res['items'][0]['snippet'].keys()

NameError: name 'res' is not defined

In [87]:
info

{u'etag': u'"iDqJ1j7zKs4x3o3ZsFlBOwgWAHU/4_eoX2xFG654PKKsiSyLzo2jVq0"',
 u'items': [{u'etag': u'"iDqJ1j7zKs4x3o3ZsFlBOwgWAHU/KfbrMhY6gSuSchJiC2vq4PjiXe0"',
   u'id': u'UCe3izJnI-GAZcWnrhlr7OYw',
   u'kind': u'youtube#channel',
   u'snippet': {u'description': u'',
    u'localized': {u'description': u'', u'title': u'gon tv'},
    u'publishedAt': u'2015-06-16T02:37:12.000Z',
    u'thumbnails': {u'default': {u'url': u'https://yt3.ggpht.com/-GS8EppGwo3Y/AAAAAAAAAAI/AAAAAAAAAAA/GgxT71FbVcw/s88-c-k-no/photo.jpg'},
     u'high': {u'url': u'https://yt3.ggpht.com/-GS8EppGwo3Y/AAAAAAAAAAI/AAAAAAAAAAA/GgxT71FbVcw/s240-c-k-no/photo.jpg'},
     u'medium': {u'url': u'https://yt3.ggpht.com/-GS8EppGwo3Y/AAAAAAAAAAI/AAAAAAAAAAA/GgxT71FbVcw/s240-c-k-no/photo.jpg'}},
    u'title': u'gon tv'}}],
 u'kind': u'youtube#channelListResponse',
 u'pageInfo': {u'resultsPerPage': 1, u'totalResults': 1}}